In [2]:
%load_ext autoreload
%autoreload 2
%run ./initializer

Path (/Users/madalenabarcelo/Desktop/BA/DMT/assignment_2/DataMiningTechniquesA2) already exists in sys.path


In [3]:
import pandas as pd
import numpy as np
from item_similarity import ClusteredItemBasedCF

In [4]:
train_df = pd.read_parquet("data/training_set_processed.parquet")
test_df = pd.read_parquet("data/test_set_processed.parquet")

In [ ]:
# Initialize model
cf_model = ClusteredItemBasedCF(k=5, n_clusters=10)


item_feature_cols = ['prop_starrating', 'prop_review_score', 'prop_location_score1']

# Only keep one row per prop_id with its features
item_features_df = train_df[['prop_id'] + item_feature_cols].drop_duplicates('prop_id').dropna()

# Handle bad values
item_features_df.replace([np.inf, -np.inf], np.nan, inplace=True)  # Replace inf
item_features_df.dropna(inplace=True)  # Drop rows with NaNs

# Sample a portion of training data for efficiency
sampled_train_df = train_df.sample(frac=0.01, random_state=42)

# Set up item-to-cluster mapping
#clusters = cf_model.cluster_items(item_features_df, features=item_feature_cols)
#cf_model.item_to_cluster = dict(zip(clusters['prop_id'], clusters['cluster']))

# Format and train model
cf_model.format_and_train(sampled_train_df, item_features=item_features_df, feature_cols=item_feature_cols)

# === 4. Predict scores for test users ===
pred_df = cf_model.predict_for_test_users(test_df)

# === 5. Sort predictions by score ===
pred_df = pred_df.sort_values(by=['srch_id', 'score'], ascending=[True, False])

# export csv for submission
pred_df[["srch_id", "prop_id"]].to_csv("submission.csv", index=False)